In [16]:
import os

import numpy as np
import torch
from lib.config import cfg

from lib.dataset.dataset2D import Dataset2D
from lib.dataset.dataset3D import Dataset3D
from lib.hybridnet.modules.efficienttrack.efficienttrack import EfficientTrack
import lib.hybridnet.modules.efficienttrack.darkpose as darkpose

from lib.config.project_manager import ProjectManager

project = ProjectManager()
project.load('Ralph_Center_Test')

#project.create_new(
#    name = 'CSV Creation',
#    dataset2D_path = '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen',
#    dataset3D_path = '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen')

cfg = project.get_cfg()

Successfully loaded project Ralph_Center_Test!


In [7]:
training_set = Dataset3D(cfg, set='train')
val_set = Dataset3D(cfg, set='val')
calibPaths = [training_set.coco.dataset['calibration']['intrinsics'], 
              training_set.coco.dataset['calibration']['extrinsics']]


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [8]:
# import lib.hybridnet.modules.efficienttrack.darkpose as darkpose
import cv2
import matplotlib.pyplot as plt

weights = "/home/timo/Desktop/HybridNet/projects/Ralph_Center_Test/models/efficienttrack/EfficientTrack-d0_210.pth"

efficientTrack = EfficientTrack('inference', cfg, weights)

In [ ]:
import cv2
import csv
import matplotlib.pyplot as plt
import itertools
from joblib import Parallel, delayed

    

video_paths = [
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_B.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LBB.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LBT.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LC.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LFB.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LFT.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RBB.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RBT.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RC.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RFB.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RFT.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_T.avi', 
]

caps = []
outs = []
for path in video_paths:
    caps.append(cv2.VideoCapture(path))
    outs.append(cv2.VideoWriter(os.path.join('/home/timo/Desktop/HybridNet/Videos_Ralph_Center', path.split('/')[-1]), cv2.VideoWriter_fourcc('M','J', 'P', 'G'), 100, (1280,1024)))

for cap in caps:
        cap.set(1,0) #99640
        
def process(cap):
    ret, img = cap.read()
    return img
        
counter = 0
    
ret = True
while ret:
    if counter % 20 == 0:
        print (counter)
    counter += 1
    imgs_orig = []
    centerHMs = []
    camsToUse = []

    imgs_orig = Parallel(n_jobs=12, require='sharedmem')(delayed(process)(cap) for cap in caps)
    imgs = torch.zeros(12,3,256,320)
    for i,img in enumerate(imgs_orig[:]):
        img = ((cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)/255.0-cfg.DATASET.MEAN)/cfg.DATASET.STD)
        img = cv2.resize(img, (320,256))
        imgs[i] = torch.from_numpy(cv2.resize(img, (320,256)).transpose(2,0,1))
    
    imgs = imgs.cuda()
    outputs = efficientTrack.model(imgs)
    preds, maxvals = darkpose.get_final_preds(outputs[1].clamp(0,255).detach().cpu().numpy(), None)
    camsToUse = []
    
    for i,val in enumerate(maxvals[:]):
        if val > 180:
            camsToUse.append(i)
    
    if len(camsToUse) >= 2:
        center3D = torch.from_numpy(val_set.reproTool.reconstructPoint((preds.reshape(12,2)*8).transpose(), camsToUse))
        reproPoints = val_set.reproTool.reprojectPoint(center3D)
        
        errors = []
        errors_valid = []
        for i in range(12):
            if maxvals[i] > 180:
                errors.append(np.linalg.norm(preds.reshape(12,2)[i]*8-reproPoints[i]))
                errors_valid.append(np.linalg.norm(preds.reshape(12,2)[i]*8-reproPoints[i]))
            else:
                errors.append(0)
        medianError = np.median(np.array(errors_valid))
        print ("Error: ", medianError)
        print ("Var based: ", medianError+2*np.sqrt(np.var(errors_valid)),medianError*4)
        camsToUse = []
        for i,val in enumerate(maxvals[:]):
            if val > 180 and errors[i] < 1.5*medianError:
                camsToUse.append(i)
        center3D = torch.from_numpy(val_set.reproTool.reconstructPoint((preds.reshape(12,2)*8).transpose(), camsToUse))
        reproPoints = val_set.reproTool.reprojectPoint(center3D)

        for i in range(12):
            colors = [(255,0,0), (255,0,0),(255,0,0),(255,0,0),(0,255,0),(0,255,0),(0,255,0),(0,255,0),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(255,255,0),(255,255,0),(255,255,0), (255,255,0),
                              (0,255,255),(0,255,255),(0,255,255),(0,255,255), (255,0,255),(100,0,100),(100,0,100)]

            cv2.circle(imgs_orig[i], (int(reproPoints[i][0]), int(reproPoints[i][1])), 2, (255,100,100), thickness=5)
            for j in range(12):
                if j in camsToUse:
                    cv2.circle(imgs_orig[i], (100+j*90,200), int(errors[j]), (255,0,0), thickness=5)
                else:
                    cv2.circle(imgs_orig[i], (100+j*90,200), int(5), (100,100,100), thickness=5)
            #cv2.putText(imgs_orig[i],str(error/12),(10,500), font, 4, (255, 0, 0), 2, cv2.LINE_AA)
            heatmap = outputs[1][i].clamp(0,255).detach().transpose(2,0).transpose(0,1).cpu().int().numpy().astype(np.uint8)
            heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
            added_image = cv2.addWeighted(imgs_orig[i],1.0, cv2.resize(heatmap, (1280,1024)),0.25,0)
            outs[i].write(added_image)
    

    #hetamap = cv2.resize(outputs[1].clamp(0,255).detach().cpu().numpy()[11][0]/255., (320,256), interpolation=cv2.cv2.INTER_NEAREST)
    #plt.imshow(hetamap, alpha=0.6)
    #plt.show()
    

0
Error:  7.867743817406721
Var based:  12.752083725270218 31.470975269626884
Error:  7.248181419521416
Var based:  14.239918756150221 28.992725678085662
Error:  6.070330505844897
Var based:  9.780390328313818 24.281322023379587
Error:  5.5551275417794574
Var based:  8.784238764117879 22.22051016711783
Error:  4.905079736235283
Var based:  8.780240569329322 19.620318944941133
Error:  6.1519115669825
Var based:  14.815337704717008 24.60764626793
Error:  4.448429355222599
Var based:  7.901067296923555 17.793717420890395
Error:  4.3393012735855185
Var based:  7.859545820444847 17.357205094342074
Error:  5.315012194210951
Var based:  11.281227480301881 21.260048776843803
Error:  6.053547883851834
Var based:  11.565859152432843 24.214191535407338
Error:  5.778068346217128
Var based:  10.234271596593295 23.112273384868512
Error:  4.133728010131884
Var based:  7.442803187837635 16.534912040527537
Error:  5.667296921302086
Var based:  10.212859736377363 22.669187685208342
Error:  5.67342527416

Error:  8.713719139772069
Var based:  18.543221676274204 34.854876559088275
Error:  9.43533082542043
Var based:  19.38055805973859 37.74132330168172
Error:  8.774430076178575
Var based:  18.4815505385434 35.0977203047143
Error:  8.9544741261606
Var based:  18.662099648704093 35.8178965046424
Error:  9.144148132534575
Var based:  19.397790103363924 36.5765925301383
Error:  8.820665628716853
Var based:  18.033392181651827 35.28266251486741
Error:  9.309083370292695
Var based:  19.128262818275168 37.23633348117078
Error:  9.0780534385428
Var based:  19.145832394230446 36.3122137541712
Error:  8.895418049507246
Var based:  18.383946044407573 35.58167219802898
Error:  9.387166900060308
Var based:  19.180446390016193 37.54866760024123
Error:  9.446738911235355
Var based:  19.349217877362406 37.78695564494142
120
Error:  8.05853846079312
Var based:  18.1079072901648 32.23415384317248
Error:  7.943603581639009
Var based:  17.473500460423807 31.774414326556037
Error:  7.791541460772136
Var base

In [ ]:
import cv2
import csv
import matplotlib.pyplot as plt
import itertools
from joblib import Parallel, delayed

    

video_paths = [
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_B.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LBB.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LBT.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LC.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LFB.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_LFT.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RBB.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RBT.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RC.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RFB.avi',
    '/media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_RFT.avi',
    '//media/timo/Elements/Recordings/Ralph/Ralph_Test_15072021/Camera_T.avi', 
]

caps = []
outs = []
for path in video_paths:
    caps.append(cv2.VideoCapture(path))
    outs.append(cv2.VideoWriter(os.path.join('/home/timo/Desktop/HybridNet/Videos_Ralph_Center', path.split('/')[-1]), cv2.VideoWriter_fourcc('M','J', 'P', 'G'), 100, (1280,1024)))

for cap in caps:
        cap.set(1,0) #99640
        
def process(cap):
    ret, img = cap.read()
    return img
        
counter = 0
    
ret = True
while ret:
    if counter % 20 == 0:
        print (counter)
    counter += 1
    imgs_orig = []
    centerHMs = []
    camsToUse = []

    imgs_orig = Parallel(n_jobs=12, require='sharedmem')(delayed(process)(cap) for cap in caps)
    imgs = torch.zeros(12,3,256,320)
    for i,img in enumerate(imgs_orig[:]):
        img = ((cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)/255.0-cfg.DATASET.MEAN)/cfg.DATASET.STD)
        img = cv2.resize(img, (320,256))
        imgs[i] = torch.from_numpy(cv2.resize(img, (320,256)).transpose(2,0,1))
    
    imgs = imgs.cuda()
    outputs = efficientTrack.model(imgs)
    preds, maxvals = darkpose.get_final_preds(outputs[1].clamp(0,255).detach().cpu().numpy(), None)
    camsToUse = []
    print (maxvals[0])
    for i,val in enumerate(maxvals):
        if val > 200:
            camsToUse.append(i)
    
    print (preds.shape)
    
    print ("Error: ", error/12)
    
    if len(camsToUse) >= 2:
        center3D = torch.from_numpy(val_set.reproTool.reconstructPoint((preds.reshape(12,2)*8).transpose(), camsToUse))
        center3D = center3D.int()
        reproPoints = val_set.reproTool.reprojectPoint(center3D)
        error = 0
        for i in range(12):
            if maxvals[i] > 200:
                error += np.linalg.norm(preds.reshape(12,2)[i]*8-reproPoints[i])
        print (reproPoints[0])
        for i in range(12):
            colors = [(255,0,0), (255,0,0),(255,0,0),(255,0,0),(0,255,0),(0,255,0),(0,255,0),(0,255,0),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(255,255,0),(255,255,0),(255,255,0), (255,255,0),
                              (0,255,255),(0,255,255),(0,255,255),(0,255,255), (255,0,255),(100,0,100),(100,0,100)]

            #cv2.circle(imgs_orig[i], (int(reproPoints[i][0]), int(reproPoints[i][1])), 2, (255,100,100), thickness=5)
            #cv2.circle(imgs_orig[i], (200,200), int(error/12), (255,0,0), thickness=5)#
            heatmap = outputs[1][i].clamp(0,255).detach().transpose(2,0).transpose(0,1).cpu().int().numpy().astype(np.uint8)
            heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
            added_image = cv2.addWeighted(imgs_orig[i],1.0, cv2.resize(heatmap, (1280,1024)),0.3,0)
            #cv2.putText(imgs_orig[i],str(error/12),(10,500), font, 4, (255, 0, 0), 2, cv2.LINE_AA)
            outs[i].write(added_image)
            font = cv2.FONT_HERSHEY_SIMPLEX
    

    #hetamap = cv2.resize(outputs[1].clamp(0,255).detach().cpu().numpy()[11][0]/255., (320,256), interpolation=cv2.cv2.INTER_NEAREST)
    #plt.imshow(hetamap, alpha=0.6)
    #plt.show()
    

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm

figure = plt.figure()
axes = figure.gca(projection='3d')

c = ['r', 'r','r','r','b','b','b','b','g','g','g','g', 'orange', 'orange','orange','orange', 'y','y','y','y','purple', 'purple','purple']
line_idxs = [[0,1], [1,2], [2,3], [4,5], [5,6], [6,7], [8,9], [9,10], [10,11], [12,13], [13,14], [14,15], [16,17], [17,18], [18,19], [3,7], [7,11], [11,15], [3,21], [7,21],[11,22], [15,22],[21,22], [18,15], [19,22]]
points3D = points3D_net[0].cpu().numpy()
center3D = center3D.cpu()
for i, point in enumerate(points3D):
    #print ("Classic:", i, point)
    if i != 20:
        axes.scatter(point[0], point[1], point[2], color = c[i])
for line in line_idxs:
    axes.plot([points3D[line[0]][0], points3D[line[1]][0]], [points3D[line[0]][1], points3D[line[1]][1]], [points3D[line[0]][2], points3D[line[1]][2]], c = 'gray')

axes.set_xlim3d(center3D[0]-100, center3D[0]+100)
axes.set_ylim3d(center3D[1]-100, center3D[1]+100)
axes.set_zlim3d(center3D[2]-100, center3D[2]+100)
plt.subplots_adjust(left=0., right=1., top=1., bottom=0.)
#plt.gca().invert_zaxis()
plt.savefig('test.png')
plt.show()